This notebook sweeps the SMU Keithley 2450, based on Joe's dummy instruments demo

In [1]:
import time
import numpy as np
import qcodes as qc
from qcodes.dataset.measurements import Measurement
from qcodes.instrument_drivers.tektronix.Keithley_2450 import Keithley_2450
from qcodes.tests.instrument_mocks import DummyInstrument

Now we will create an instance of the class Keithley

In [2]:
Keith = Keithley_2450('Keith', 'USB0::0x05E6::0x2450::04358606::INSTR')

First a test to see that the Keithley 2450 works

In [3]:
Keith.rangev(20)
Keith.volt(1)
Keith.curr()
#x=Keith.sense()
dac.ch1.step = 0.1
dac.ch1.inter_delay = 0.001

-1.268326E-12


Start a new experiment that will go in the current database

In [4]:
qc.new_experiment(name='demotest3', sample_name='my best sample3')

demotest3#my best sample3#18@./experiments.db
---------------------------------------------

Class Sweep to sweep and then plot data, should work for any intrument

In [5]:
# %load sweep.py
import io
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import qcodes as qc
from qcodes.dataset.measurements import Measurement
from IPython import display


class Sweep(object):
    def __init__(self):
        self._params = []
    
    def follow_param(self, p):
        self._params.append(p)

    def _create_measurement(self, *set_params):
        meas = Measurement()
        for p in set_params:
            meas.register_parameter(p)
        meas.register_custom_parameter('time', label='Time', unit='s')
        for p in self._params:
            meas.register_parameter(p, setpoints=(*set_params, 'time',))
        return meas
    
    def sweep(self, set_param, vals, inter_delay=None):
        if inter_delay is not None:
            d = len(vals)*inter_delay
            h, m, s = int(d/3600), int(d/60) % 60, int(d) % 60
            print(f'Minimum duration: {h}h {m}m {s}s')

        fig = plt.figure(figsize=(4*(2 + len(self._params)),4))
        grid = plt.GridSpec(4, 1 + len(self._params), hspace=0)
        setax = fig.add_subplot(grid[:, 0])
        setax.set_xlabel('Time (s)')
        setax.set_ylabel(f'{set_param.label} ({set_param.unit})')
        setaxline = setax.plot([], [])[0]

        paxs = []
        plines = []
        for i, p in enumerate(self._params):
            ax = fig.add_subplot(grid[:, 1 + i])
            ax.set_xlabel(f'{set_param.label} ({set_param.unit})')
            ax.set_ylabel(f'{p.label} ({p.unit})')
            paxs.append(ax)
            plines.append(ax.plot([], [])[0])

        laxs = []
        llines = []


        fig.tight_layout()
        fig.show()

        meas = self._create_measurement(set_param)
        with meas.run() as datasaver:
            try:
                t0 = time.monotonic()
                for setpoint in vals:
                    t = time.monotonic() - t0
                    set_param.set(setpoint)

                    setaxline.set_xdata(np.append(setaxline.get_xdata(), t))
                    setaxline.set_ydata(np.append(setaxline.get_ydata(), setpoint))
                    setax.relim()
                    setax.autoscale_view()

                    if inter_delay is not None:
                        plt.pause(inter_delay)

                    data = [
                        (set_param, setpoint),
                        ('time', t)
                        ]
                    for i, p in enumerate(self._params):
                        v = p.get()
                        data.append((p, v))
                        plines[i].set_xdata(np.append(plines[i].get_xdata(), setpoint))
                        plines[i].set_ydata(np.append(plines[i].get_ydata(), v))
                        paxs[i].relim()
                        paxs[i].autoscale_view()

                    datasaver.add_result(*data)

                    fig.tight_layout()
                    fig.canvas.draw()
                    plt.pause(0.001)

                d = time.monotonic() - t0
                h, m, s = int(d/3600), int(d/60) % 60, int(d) % 60
                print(f'Completed in: {h}h {m}m {s}s')

                b = io.BytesIO()
                fig.savefig(b, format='png')
                display.display(display.Image(data=b.getbuffer(), format='png'))
            except KeyboardInterrupt:
                print('At stoppage')
                for i,p in enumerate(data): 
                    print(data[i][0],'is equal to',data[i][1])                   
                pass

This sets data to be plotted in a separate window from the notebook

In [ ]:
%matplotlib qt

Sets an instance of the class sweep <br>
Adds the parameter to measure <br>
Runs the function sweep with voltages as the sweep parameter

In [54]:
s = Sweep()
s.follow_param(Keith.curr)
s.sweep(Keith.volt, np.linspace(0, 1, 100), inter_delay=.01)

Minimum duration: 0h 0m 1s
Starting experimental run with id: 144
3.378654E-12
2.305942E-12
2.557239E-12
2.743102E-12
2.602478E-12
2.498765E-12
2.290627E-12
3.251961E-12
3.051480E-12
2.070655E-12
2.937317E-12
2.576034E-12
2.642165E-12
3.208795E-12
2.009396E-12
2.657478E-12
3.167731E-12
1.458074E-12
3.020140E-12
2.504335E-12
1.877112E-12
1.884097E-12
2.924086E-12
2.927572E-12
2.960289E-12
3.455666E-12
5.698323E-13
3.984247E-12
2.055309E-12
-7.553125E-12
1.452472E-12
3.978648E-12
7.729632E-12
2.788583E-13
6.296982E-13
1.872263E-12
3.223367E-12
5.199374E-12
5.438162E-12
2.222993E-12
1.406955E-12
1.689714E-12
4.467127E-12
3.645181E-12
2.189414E-12
2.805549E-12
8.774281E-13
1.595905E-12
1.721207E-12
1.650898E-12
At stoppage
Keith_volt is equal to 0.505050505051
time is equal to 40.85899999999674


In [ ]:
Now we have created 

In [7]:
qc.dataset.experiment_container.experiments()

[demo#my best sample#1@./experiments.db
 --------------------------------------
 1-results-1-dac_ch1,time,dmm_ig,dmm_idc-10
 2-results-2-dac_ch1,dac_ch2,dmm_ig-100
 3-results-3-dac_ch1,time,dmm_ig,dmm_idc-10,
 demo#my best sample#2@./experiments.db
 --------------------------------------
 4-results-1-dac_ch1,time,dmm_ig,dmm_idc-10
 5-results-2-x,y-10
 6-results-3-dac_ch1,time,dmm_ig,dmm_idc-10
 7-results-4-dac_ch1,time,dmm_ig,dmm_idc-10
 8-results-5-dac_ch1,time,dmm_ig,dmm_idc-10
 9-results-6-dac_ch1,time,dmm_ig,dmm_idc-10
 10-results-7-x,y-10
 11-results-8-dac_ch1,time,dmm_ig,dmm_idc-10
 12-results-9-dac_ch1,time,dmm_ig,dmm_idc-10
 13-results-10-dac_ch1,time,dmm_ig,dmm_idc-10
 14-results-11-dac_ch1,time,dmm_ig,dmm_idc-10
 15-results-12-dac_ch1,time,dmm_ig,dmm_idc-20
 16-results-13-dac_ch1,time,dmm_ig,dmm_idc-10
 17-results-14-dac_ch1,time,dmm_ig-10
 18-results-15-x,y-10
 19-results-16-dac_ch1,time,dmm_ig,dmm_idc-10
 20-results-17-dac_ch1,time,dmm_ig,dmm_idc-10
 21-results-18-dac_ch1,t

In [8]:
ex = qc.dataset.experiment_container.load_experiment(10)

In [24]:
from qcodes.dataset.data_export import get_data_by_id 
fii = get_data_by_id(ex.data_sets()[2].run_id)

In [25]:
fii

[[{'name': 'dac_ch1',
   'label': 'Gate ch1',
   'unit': 'V',
   'data': array([  0.        ,   2.77777778,   5.55555556,   8.33333333,
           11.11111111,  13.88888889,  16.66666667,  19.44444444,
           22.22222222,  25.        ])},
  {'name': 'time',
   'label': 'Time',
   'unit': 's',
   'data': array([  0.   ,   1.438,   2.641,   3.828,   5.016,   6.203,   7.391,
            8.594,   9.813,  11.   ])},
  {'name': 'Keith_curr',
   'label': 'Current',
   'unit': 'A',
   'data': array([  9.85979200e-07,   9.85890800e-07,   9.85876700e-07,
            9.85872000e-07,   9.85877500e-07,   9.85861900e-07,
            9.85862400e-07,   9.85868000e-07,   9.85825400e-07,
            9.85881000e-07])}]]

In [52]:
fii[0][1]['data']

array([  0.   ,   1.438,   2.641,   3.828,   5.016,   6.203,   7.391,
         8.594,   9.813,  11.   ])